In [1]:
%reload_ext autoreload
%autoreload 2
import json
import logging
import numpy as np
import pandas as pd
from pyeed import Pyeed
import matplotlib.pyplot as plt

from pyeed.analysis.ontology_loading import OntologyAdapter
from pyeed.analysis.embedding_analysis import EmbeddingTool
from pyeed.analysis.sequence_alignment import PairwiseAligner
from pyeed.analysis.mutation_detection import MutationDetection

/home/nab/anaconda3/envs/pyeed_niklas/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
LOGGER = logging.getLogger(__name__)

In [ ]:
et = EmbeddingTool()
pa = PairwiseAligner()
md = MutationDetection()

In [ ]:
uri = "bolt://127.0.0.1:1123"
user = "neo4j"
password = "niklasonlytems"

# Create a Pyeed object, automatically connecting to the database
eedb = Pyeed(uri, user, password)

In [ ]:
# For testing purposes, we will wipe the database and remove all constraints
# eedb.db.wipe_database(date='2024-12-13')
# eedb.db.remove_db_constraints(user=user, password=password)

# DB connector is an attribute of the Pyeed object, type `DatabaseConnector`
LOGGER.info(f"Database stats: {eedb.db.stats()}")

# The first time the pyeed database is initialized, we need to create the constraints which are defined in the pyeed graph model
eedb.db.initialize_db_constraints(user=user, password=password)

In [ ]:
# read in the pandas dataframe
df = pd.read_csv('/home/nab/Niklas/TEM-lactamase/data/002_combined_data/TEM_lactamase.csv', sep=';')
print(df.head())

In [ ]:
from pyeed.analysis.standard_numbering import StandardNumberingTool

# Apply the standard numbering
standard_numbering = StandardNumberingTool(name="test_standard_numbering_all")
standard_numbering.apply_standard_numbering(base_sequence_id='AAP20891.1', db=eedb.db, list_of_seq_ids=df['protein_id_database'].tolist())

In [ ]:
# now we want to start with a mutational detection
# a first approach is to just include the 209 TEMs and see if we can detect the mutations
# here we find the colsest neighbor based on the standard numbering and then we can find their mutations
# we also want to coun the number of mutations, the idendeity, the cosine distance and the euclidean distance between all of them
# we can therefore perform a pairwise alignment between the found neighbours

# we first need to find the closest neighbour to the base sequence
n_neighbours = 40000

# count the number of pairwise alignments performed
# we want to expect 209*209 / 2 = 21801 pairwise alignments
counter = 0
already_processed_pairs = []

# iterate over the different proteins ids in df
for index, row in df.iterrows():
    print(f"Processing protein {index+1} of {len(df)} with a db id of {row['protein_id_database']}")
    if pd.isna(row['protein_id_database']):
        print(f"Skipping protein {index+1} of {len(df)} because it does not have a database id")
        continue
    # get the id in the database
    base_sequence_id = row['protein_id_database']

    closest_neighbours = et.find_closest_matches_simple(start_sequence_id=base_sequence_id, db=eedb.db, n = n_neighbours)
    # print(f"The number of closest neighbours is: {len(closest_neighbours)}")

    # the protein itself is returned as well
    # the list is build up of tuples with the following structure: (sequence_id, distance)
    closest_neighbours_ids = [neighbour[0] for neighbour in closest_neighbours]
    # print(f"The closest neighbours ids are: {closest_neighbours_ids}")

    # for the moment we only want to look at ids which are in the TEM-209 list
    # this list is stored in the df dataframe
    # we can get the ids from the df dataframe by using the 'protein_id_database' column
    # we need to make sure that the ids are in the closest_neighbours_ids list
    # we can do this by using the intersection of the two lists
    tem_209_ids = df['protein_id_database'].dropna().tolist()
    # print(f"The TEM-209 ids are: {tem_209_ids}")

    # now we can get the intersection of the two lists
    intersection = list(set(closest_neighbours_ids) & set(tem_209_ids))
    # print(f"The intersection of the two lists is: {len(intersection)}")

    # we need to create all of the permutations of the neighbours with the base sequence
    # please that the reverse direction should not be included
    # this means that the base sequence is always the first element in the tuple and the second element is the neighbour
    permutations = [(base_sequence_id, neighbour) for neighbour in intersection]
    # print(f"The permutations of the neighbours including the base sequence are: {len(permutations)}")

    # we now want to exclude the pairs that we already processed keeping in mind that we always add in the list both directions
    permuations_to_process = [pair for pair in permutations if pair not in already_processed_pairs]

    # we now update the already_processed_pairs list with the new pairs
    # we need to add the reverse of the pair as well
    already_processed_pairs.extend([(pair[1], pair[0]) for pair in permuations_to_process])
    already_processed_pairs.extend(permuations_to_process)
    
    # now we run a pairwise alignment between the found neighbours
    pairwise_alignment = pa.align_multipairwise(ids=intersection, db=eedb.db, pairs = permuations_to_process)
    # print(f"The pairwise alignment between the found neighbours and the base sequence is: {pairwise_alignment}")
    counter += len(permuations_to_process)

    # now we detect the mutations
    mutations = []
    # now we can detect the mutations
    for i in range(len(permuations_to_process)):
        if permuations_to_process[i][0] == permuations_to_process[i][1]:
            # print(f"Skipping permutation {i+1} of {len(permuations_to_process)} because they are the same")
            continue

        # print(f"Mutation detection for permuations_to_process {i+1} of {len(permuations_to_process)} between {permuations_to_process[i][0]} and {permuations_to_process[i][1]}")
        result = md.get_mutations_between_sequences(sequence_id1=permuations_to_process[i][0], sequence_id2=permuations_to_process[i][1], db=eedb.db, save_to_db=True, standard_numbering_tool_name="test_standard_numbering")
        # print(f"Number of mutations: {len(result)}")

        mutations.append(result)



print(f"The number of pairwise alignments performed is: {counter}")
# 48min  53 sec 
# the number was 21945


In [ ]:
ids_tems = df['protein_id_database'].dropna().tolist()
intersection_number_of_tems = len(ids_tems)
print(f"The number of TEMs is: {intersection_number_of_tems}")

In [ ]:
# now we want to analyze the mutations
# we want to perform a mutational analysis on all of the mutations
# we are are intrested in creating a matrix which has all of the proteins in df as rows and columns and the values are the number of mutations between the two proteins
# from a logic standpoint this is a square matrix with 209 rows and columns the same as in the embedding_analysis.ipynb file
distance_matrix_mutations = np.zeros((intersection_number_of_tems, intersection_number_of_tems))
protein_ids_mutations = []

# now we want to create a distance matrix for the mutations
# a mutaion can appear between two proteins the direction is not important
query = """
MATCH (p1:Protein)-[r:HAS_MUTATION]-(p2:Protein)
WHERE p1.accession_id IN $ids AND p2.accession_id IN $ids
RETURN p1.accession_id AS protein1, p2.accession_id AS protein2, COUNT(r) AS mutations
"""

results_mutations = eedb.db.execute_read(query, {"ids": ids_tems})
print(results_mutations[:10])

# now we want to create a distance matrix for the mutations
# the distance matrix is a square matrix with 209 rows and columns
# the values are the number of mutations between the two proteins
# the diagonal is 0 since a protein does not mutate with itself
for i in range(len(ids_tems)):
    for j in range(len(ids_tems)):
        if i == j:
            distance_matrix_mutations[i, j] = 0
        else:
            distance_matrix_mutations[i, j] = next((record["mutations"] for record in results_mutations if record["protein1"] == ids_tems[i] and record["protein2"] == ids_tems[j]), 0)
        
    protein_ids_mutations.append(ids_tems[i])

# save the distance matrix to a numpy file
np.save("/home/nab/Niklas/TEM-lactamase/data/002_combined_data/distance_matrix_mutations.npy", distance_matrix_mutations)
np.save("/home/nab/Niklas/TEM-lactamase/data/002_combined_data/protein_ids_mutations.npy", protein_ids_mutations)